In [16]:
import regex as re
from pymongo import MongoClient
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, _preprocess

N_GRAM_LOWER_LIM = 1
N_GRAM_UPPER_LIM = 2
NEGATIVES = list(stopwords.words())
DB_CONNECTION = 'mongodb://localhost:27017'

client = MongoClient(DB_CONNECTION)
sample_review = client.course_evals.entries.find_one()

In [17]:
def tokenize(raw_text):
    pattern = re.compile('[^a-zA-z\s]+', re.UNICODE)
    text = re.sub(pattern, ' ', raw_text)
    text = re.sub('\s+', ' ', text)
    return text

In [18]:
vectorized = CountVectorizer(ngram_range=(N_GRAM_LOWER_LIM,N_GRAM_UPPER_LIM))
example = tokenize(sample_review['RESPONSE_TEXT'])
X = vectorized.fit_transform(raw_documents=[example])
vectorized.get_feature_names()

['anatomy',
 'anatomy muscles',
 'drawing',
 'drawing techniques',
 'facial',
 'facial anatomy',
 'for',
 'for skulls',
 'muscles',
 'muscles skeletal',
 'skeletal',
 'skeletal structure',
 'skulls',
 'structure',
 'structure drawing',
 'techniques',
 'techniques for']